# T1053.006 - Scheduled Task/Job: Systemd Timers
Adversaries may abuse systemd timers to perform task scheduling for initial or recurring execution of malicious code. Systemd timers are unit files with file extension <code>.timer</code> that control services. Timers can be set to run on a calendar event or after a time span relative to a starting point. They can be used as an alternative to [Cron](https://attack.mitre.org/techniques/T1053/003) in Linux environments.(Citation: archlinux Systemd Timers Aug 2020) Systemd timers may be activated remotely via the <code>systemctl</code> command line utility, which operates over [SSH](https://attack.mitre.org/techniques/T1021/004).(Citation: Systemd Remote Control)

Each <code>.timer</code> file must have a corresponding <code>.service</code> file with the same name, e.g., <code>example.timer</code> and <code>example.service</code>. <code>.service</code> files are [Systemd Service](https://attack.mitre.org/techniques/T1543/002) unit files that are managed by the systemd system and service manager.(Citation: Linux man-pages: systemd January 2014) Privileged timers are written to <code>/etc/systemd/system/</code> and <code>/usr/lib/systemd/system</code> while user level are written to <code>~/.config/systemd/user/</code>.

An adversary may use systemd timers to execute malicious code at system startup or on a scheduled basis for persistence.(Citation: Arch Linux Package Systemd Compromise BleepingComputer 10JUL2018)(Citation: gist Arch package compromise 10JUL2018)(Citation: acroread package compromised Arch Linux Mail 8JUL2018) Timers installed using privileged paths may be used to maintain root level persistence. Adversaries may also install user level timers to achieve user level persistence.

## Atomic Tests

### Atomic Test #1 - Create Systemd Service and Timer
This test creates Systemd service and timer then starts and enables the Systemd timer 

**Supported Platforms:** linux
#### Attack Commands: Run with `bash`
```bash
echo "[Unit]" > /etc/systemd/system/art-timer.service
echo "Description=Atomic Red Team Systemd Timer Service" >> /etc/systemd/system/art-timer.service
echo "[Service]" >> /etc/systemd/system/art-timer.service
echo "Type=simple" >> /etc/systemd/system/art-timer.service
echo "ExecStart=/bin/touch /tmp/art-systemd-timer-marker" >> /etc/systemd/system/art-timer.service
echo "[Install]" >> /etc/systemd/system/art-timer.service
echo "WantedBy=multi-user.target" >> /etc/systemd/system/art-timer.service
echo "[Unit]" > /etc/systemd/system/art-timer.timer
echo "Description=Executes Atomic Red Team Systemd Timer Service" >> /etc/systemd/system/art-timer.timer
echo "Requires=art-timer.service" >> /etc/systemd/system/art-timer.timer
echo "[Timer]" >> /etc/systemd/system/art-timer.timer
echo "Unit=art-timer.service" >> /etc/systemd/system/art-timer.timer
echo "OnCalendar=*-*-* *:*:00" >> /etc/systemd/system/art-timer.timer
echo "[Install]" >> /etc/systemd/system/art-timer.timer
echo "WantedBy=timers.target" >> /etc/systemd/system/art-timer.timer
systemctl start art-timer.timer
systemctl enable art-timer.timer
systemctl daemon-reload
```

In [ ]:
Invoke-AtomicTest T1053.006 -TestNumbers 1

### Atomic Test #2 - Create a user level transient systemd service and timer
Schedule a user level transient task (will not survive a reboot) without having to create the .timer or .service files by using the systemd-run command. 

**Supported Platforms:** linux
#### Dependencies:  Run with `sh`!
##### Description: Check if systemd-run exists on the machine

##### Check Prereq Commands:
```sh
if [ -x "$(command -v systemd-run)" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
echo "Install systemd on the machine."; exit 1;

```

In [ ]:
Invoke-AtomicTest T1053.006 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
systemd-run --user --unit=Atomic-Red-Team --on-calendar '*:0/1' /bin/sh -c 'echo "$(date) $(whoami)" >>/tmp/log'
```

In [ ]:
Invoke-AtomicTest T1053.006 -TestNumbers 2

### Atomic Test #3 - Create a system level transient systemd service and timer
Schedule a system level transient task (will not survive a reboot) without having to create the .timer or .service files by using the systemd-run command. 

**Supported Platforms:** linux
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Check if systemd-run exists on the machine

##### Check Prereq Commands:
```sh
if [ -x "$(command -v systemd-run)" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
echo "Install systemd on the machine."; exit 1;

```

In [ ]:
Invoke-AtomicTest T1053.006 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
systemd-run --unit=Atomic-Red-Team --on-calendar '*:0/1' /bin/sh -c 'echo "$(date) $(whoami)" >>/tmp/log'
```

In [ ]:
Invoke-AtomicTest T1053.006 -TestNumbers 3

## Detection
Systemd timer unit files may be detected by auditing file creation and modification events within the <code>/etc/systemd/system</code>, <code>/usr/lib/systemd/system/</code>, and <code>~/.config/systemd/user/</code> directories, as well as associated symbolic links. Suspicious processes or scripts spawned in this manner will have a parent process of ‘systemd’, a parent process ID of 1, and will usually execute as the ‘root’ user.

Suspicious systemd timers can also be identified by comparing results against a trusted system baseline. Malicious systemd timers may be detected by using the systemctl utility to examine system wide timers: <code>systemctl list-timers –all</code>. Analyze the contents of corresponding <code>.service</code> files present on the file system and ensure that they refer to legitimate, expected executables.

Audit the execution and command-line arguments of the 'systemd-run' utility as it may be used to create timers.(Citation: archlinux Systemd Timers Aug 2020)